In [ ]:
import sqlite3
import csv

# создание таблицы базы данных
def initialize_database():
    conn = sqlite3.connect('phonebook.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS PhoneBook (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name TEXT NOT NULL,
            phone TEXT NOT NULL UNIQUE
        )
    ''')
    conn.commit()
    conn.close()

# очистка таблицы и сброс счётчика
def reset_table():
    conn = sqlite3.connect('phonebook.db')
    cursor = conn.cursor()
    cursor.execute("DELETE FROM PhoneBook")
    cursor.execute("DELETE FROM sqlite_sequence WHERE name='PhoneBook'")
    conn.commit()
    conn.close()
    print("All entries deleted. IDs reset.")

# добавление одного контакта с консоли
def add_contact_manually():
    name = input("Enter name: ")
    phone = input("Enter phone number: ")
    conn = sqlite3.connect('phonebook.db')
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO PhoneBook (first_name, phone) VALUES (?, ?)", (name.strip(), phone.strip()))
        print("Contact saved.")
    except sqlite3.IntegrityError:
        print("This phone number already exists.")
    conn.commit()
    conn.close()

# вставка или обновление по имени
def upsert_contact(name, phone):
    conn = sqlite3.connect("phonebook.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM PhoneBook WHERE first_name = ?", (name,))
    if cursor.fetchone():
        cursor.execute("UPDATE PhoneBook SET phone = ? WHERE first_name = ?", (phone, name))
        print("Number updated.")
    else:
        try:
            cursor.execute("INSERT INTO PhoneBook (first_name, phone) VALUES (?, ?)", (name, phone))
            print("New contact added.")
        except sqlite3.IntegrityError:
            print("Phone already present in database.")
    conn.commit()
    conn.close()

# импорт данных из csv файла
def load_contacts_from_csv(filepath):
    conn = sqlite3.connect('phonebook.db')
    cursor = conn.cursor()
    try:
        with open(filepath, newline='', encoding='utf-8-sig') as file:
            reader = csv.reader(file)
            for idx, row in enumerate(reader, 1):
                if len(row) < 2:
                    print(f"Line {idx} ignored: {row}")
                    continue
                name, phone = row[0].strip(), row[1].strip()
                if not name or not phone:
                    print(f"Line {idx} skipped: {row}")
                    continue
                try:
                    cursor.execute("INSERT INTO PhoneBook (first_name, phone) VALUES (?, ?)", (name, phone))
                except sqlite3.IntegrityError:
                    print(f"Duplicate ignored: {phone}")
        print("CSV loading complete.")
    except FileNotFoundError:
        print("CSV file not found.")
    conn.commit()
    conn.close()

# пакетная вставка с проверками
def batch_add_contacts(entries):
    conn = sqlite3.connect("phonebook.db")
    cursor = conn.cursor()
    errors = []
    for name, phone in entries:
        if not phone.isdigit():
            errors.append((name, phone))
            continue
        cursor.execute("SELECT * FROM PhoneBook WHERE first_name = ?", (name,))
        if cursor.fetchone():
            cursor.execute("UPDATE PhoneBook SET phone = ? WHERE first_name = ?", (phone, name))
        else:
            try:
                cursor.execute("INSERT INTO PhoneBook (first_name, phone) VALUES (?, ?)", (name, phone))
            except sqlite3.IntegrityError:
                errors.append((name, phone))
    conn.commit()
    conn.close()
    return errors

# выборка по шаблону
def search_contacts(term):
    conn = sqlite3.connect("phonebook.db")
    cursor = conn.cursor()
    pattern = f"%{term}%"
    cursor.execute("SELECT * FROM PhoneBook WHERE first_name LIKE ? OR phone LIKE ?", (pattern, pattern))
    results = cursor.fetchall()
    conn.close()
    return results

# постраничный вывод
def fetch_with_pagination(limit, offset):
    conn = sqlite3.connect("phonebook.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM PhoneBook LIMIT ? OFFSET ?", (limit, offset))
    rows = cursor.fetchall()
    conn.close()
    return rows

# показ всех записей
def show_all():
    conn = sqlite3.connect('phonebook.db')
    cursor = conn.cursor()
    cursor.execute("SELECT first_name, phone FROM PhoneBook")
    records = cursor.fetchall()
    if records:
        print("\nAll contacts:")
        for i, (name, phone) in enumerate(records, 1):
            print(f"{i}. {name} — {phone}")
    else:
        print("PhoneBook is empty.")
    conn.close()

# удаление по значению
def remove_entry():
    criterion = input("Delete by (first_name / phone): ")
    if criterion not in ['first_name', 'phone']:
        print("Wrong option.")
        return
    val = input("Enter value to remove: ")
    remove_by_field(criterion, val)

def remove_by_field(field, val):
    conn = sqlite3.connect("phonebook.db")
    cursor = conn.cursor()
    cursor.execute(f"DELETE FROM PhoneBook WHERE {field} = ?", (val,))
    count = cursor.rowcount
    conn.commit()
    conn.close()
    if count:
        print(f"{count} entry(ies) removed.")
    else:
        print("No matching records found.")

# меню приложения
def start_menu():
    initialize_database()
    while True:
        print("\nPHONEBOOK SYSTEM")
        print("1. Add new contact manually")
        print("2. Upload from CSV")
        print("3. Update contact by name")
        print("4. Display all entries")
        print("5. Remove contact")
        print("6. Wipe phonebook")
        print("7. Search contacts")
        print("8. Upsert contact")
        print("9. Mass insert")
        print("10. View with pagination")
        print("11. Delete by specific field")
        print("0. Exit")

        option = input("Choose an action: ")

        if option == "1":
            add_contact_manually()
        elif option == "2":
            path = input("CSV filename: ")
            load_contacts_from_csv(path)
        elif option == "3":
            print("Use upsert option to update.")
        elif option == "4":
            show_all()
        elif option == "5":
            remove_entry()
        elif option == "6":
            reset_table()
        elif option == "7":
            key = input("Search term: ")
            found = search_contacts(key)
            if found:
                for i, row in enumerate(found, 1):
                    print(f"{i}. {row[1]} — {row[2]}")
            else:
                print("No matches.")
        elif option == "8":
            nm = input("Name: ")
            ph = input("Phone: ")
            upsert_contact(nm, ph)
        elif option == "9":
            num = int(input("Number of entries: "))
            data = [(input(" Name: "), input(" Phone: ")) for _ in range(num)]
            bad_entries = batch_add_contacts(data)
            if bad_entries:
                print("Invalid entries:")
                for e in bad_entries:
                    print(f" - {e}")
            else:
                print("All records added.")
        elif option == "10":
            l = int(input("Limit: "))
            o = int(input("Offset: "))
            records = fetch_with_pagination(l, o)
            for i, row in enumerate(records, o + 1):
                print(f"{i}. {row[1]} — {row[2]}")
        elif option == "11":
            field = input("Field (first_name / phone): ")
            val = input("Value to remove: ")
            remove_by_field(field, val)
        elif option == "0":
            print("Goodbye!")
            break
        else:
            print("Unknown selection.")

if __name__ == "__main__":
    start_menu()
